# Direct Engine Access

This notebook is about playing around to create an entry point where the user manages their state themselves.

In [ ]:
import guidance
from huggingface_hub import hf_hub_download

gguf = hf_hub_download(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="Phi-3-mini-4k-instruct-q4.gguf",
)

# Define the model we will use
# lm = guidance.models.LlamaCpp(gguf, n_gpu_layers=-1)
lm = guidance.models.Transformers("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
messages = [
    {
        "role" : "user",
        "content": "Tell me about yourself"
    }
]

In [ ]:
grammar = """%llguidance {}

start: "My name is " name " and my motto is " motto
name[capture="name", temperature=1.0]: NAME
motto[capture="motto", temperature=0.7]: MOTTO
NAME: "Phi-3, the Magnificent"
    | "Phi-3, the Terrible"
    | "Phi-3, the Great"
    | "Phi-3, the Conqueror"
MOTTO: "Look on my works ye mighty, and despair"
    | "Apres moi, le deluge"
    | "Alea iacta est"
    | "Apres moi, le table"
"""

Grab the `engine` object out of the Model. Note that this is only going to work for local models (basically, Models contain Interpreters, but only the local Interpreters then contain an engine)

In [ ]:
engine = lm._interpreter.engine

Call the new API

In [ ]:
engine.chat_completion(messages, grammar)